In [1]:
import aitk.robots
aitk.robots.__version__

'0.8.0'

This program allows the robot to wander around an empty world.
It uses a Grid imposed on the World to track the robot's location
over time. It plots a heat map of the locactions visted and computes 
the pecentage of Grid locations visited.

In [2]:
world = aitk.robots.World(width=200, height=200, scale=5.0)
robot = aitk.robots.Scribbler(x=100, y=100, a=87, max_trace_length=60)
robot.add_device(aitk.robots.RangeSensor(position=(6,-6),max=20,a=0,width=57.3,name="left-ir"))
robot.add_device(aitk.robots.RangeSensor(position=(6,6),max=20,a=0,width=57.3,name="right-ir"))
world.add_robot(robot)
world.update()
world.save()

Random seed set to: 1593568


In [3]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [4]:
from math import floor

class Grid(object):
    """This class creates a grid of locations on top of a simulated world
    to monitor how much of the world has been visited. Each grid location
    is initally set to 0 to indicate that it is unvisited, and is updated
    to 1, once it has been visited."""
    def __init__(self, grid_width, world_width):
        self.grid_width = grid_width
        self.world_width = world_width
        self.grid = []
        for i in range(self.grid_width):
            self.grid.append([0] * self.grid_width)

    def show(self):
        """Print a representation of the grid."""
        for i in range(self.grid_width - 1, -1, -1):
            for j in range(self.grid_width):
                print("%3d" % self.grid[i][j], end=" ")
            print()
        print()
        
    def update(self, x, y):
        """In the simulator, the origin is at the top-left corner.
        Update the appropriate grid location."""
        size = self.world_width/self.grid_width
        col = floor(x/size)
        row = int(self.grid_width) - floor(y/size) - 1
        self.grid[row][col] += 1
        
    def analyze_visits(self):
        """Calculate the percentage of visited cells in the grid."""
        cells_visited = 0
        for i in range(self.grid_width):
            for j in range(self.grid_width):
                if self.grid[i][j] > 0:
                    cells_visited += 1
        percent_visited = cells_visited/self.grid_width**2
        return percent_visited

In [5]:
from random import random

def wander(robot):
    left = robot[0].get_distance()
    right = robot[1].get_distance()
    x, y, direction = robot.get_pose()
    robot.state["grid"].update(x, y)
    # Save the robot's location only once per second
    if left == robot[0].get_max() and right == robot[1].get_max() and \
        robot.state["timer"] == 0:
        #front clear, move random direction
        direction = 1
        if random() < 0.5:
            direction = -1
        robot.move(0.5, direction*random())
        if robot.state["debug"]: robot.speak("F")
    elif robot.state["timer"] > 0 and robot.state["timer"] < 5:
        #timer triggered, continue current rotation
        robot.state["timer"] += 1
        if robot.state["debug"]: robot.speak("timer %d" % (robot.state["timer"]))
    elif left < robot[0].get_max():
        #obstacle on left, turn right, trigger timer
        robot.move(0, -0.4)
        robot.state["timer"] = 1
        if robot.state["debug"]: robot.speak("R")
    elif right < robot[1].get_max():
        #obstacle on right, turn left, trigger timer
        robot.move(0, 0.4)
        robot.state["timer"] = 1
        if robot.state["debug"]: robot.speak("L")
    else:
        #reset timer to zero
        robot.state["timer"] = 0
        if robot.state["debug"]: robot.speak("reset")    
    

In [6]:
from random import randrange

robot.state["timer"] = 0
robot.state["grid"] = Grid(10, 200)
robot.state["debug"] = True

world.reset()

world.set_seed(randrange(1,100)) # allow randomness

world.run([wander]) # will run until you interrupt the kernel
#world.seconds(60, [wander], real_time=False)

Using random seed: 1593568
Using random seed: 57


0it [00:00, ?it/s]

Simulation stopped at: 00:00:44.0; speed 0.98 x real time


In [7]:
g = robot.state["grid"]
g.show()
g.analyze_visits()

  0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0  17  20   4   0 
  0   0   0   0  13  25   5   0  30   0 
  0   0   0   0  22   0   0   0  20   0 
  0   0   0   0  21  10   0   0  33   0 
  0   0   0   0   0   1   0  19   6   0 
  0   0   0   0   0   0   0  23  26   0 
  0   0   0   0   0   0   0   7  84   0 
  0   0   0   0   0   0   0   0  54   0 
  0   0   0   0   0   0   0   0   0   0 



0.2

In [8]:
from bqplot import pyplot as plt
from bqplot import ColorScale
from ipywidgets import Layout
import numpy as np

fig = plt.figure(
    title='Coverage',
    layout=Layout(width='650px', 
                  height='650px'),
)
plt.scales(scales={'color': ColorScale(scheme='Greys', reverse=True)})
heatmap = plt.heatmap(np.array(g.grid))
fig

Figure(axes=[ColorAxis(scale=ColorScale(reverse=True, scheme='Greys')), Axis(scale=LinearScale()), Axis(orient…